1. How to run an inelastic traffic assignment on a real network

In [ ]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
import os
import sys
sys.path.append("../../..")
import pandas as pd
import geopandas as gpd
import numpy as np
from dyntapy.supply_data import road_network_from_place, relabel_graph
from dyntapy.demand_data import add_centroids, od_graph_from_matrix
from dyntapy.assignments import StaticAssignment
from dyntapy.visualization import show_network
from pyproj import Proj, transform
from pickle import dump

In [ ]:
# fill in these parameters 
# IMPORTANT: Use the same parameter values for the buffer as was done in STA_prep_script!
ext = 0  # 0 or 2
buffer = 40
city = 'BRUSSEL'

HERE = os.path.dirname(os.path.realpath("__file__"))
data_path = HERE + os.path.sep + os.pardir + os.path.sep + os.pardir + os.path.sep + 'data_map' + os.path.sep + 'STA'
zoning_path = data_path + os.path.sep + 'shapefiles' + os.path.sep + city + "_" + str(buffer) + "_10" + os.path.sep + city + "_" + str(buffer) + "_10.shp"
if ext == 2:
    od_path = data_path + os.path.sep + 'OD_matrix' + os.path.sep + city + "_ext_tr_" + str(buffer) + "_9_10.xlsx"
else: 
    od_path = data_path + os.path.sep + 'OD_matrix' + os.path.sep + city + "_" + str(buffer) + "_9_.xlsx"
print(zoning_path)


In [ ]:
# Creating the network for Brussels. 
network = road_network_from_place(city, buffer_dist_close=buffer*1000)
network = relabel_graph(network)
show_network(network,notebook=True)


In [ ]:
zoning = gpd.read_file(zoning_path)
od = pd.read_excel(od_path)
od_array = od.to_numpy() # The OD matrix is now stored in a numpy array

# Retrieve zone number, x_centroid (LON) and y_centroid (LAT) from each zone. 
zone_numbers = zoning["ZONENUMMER"]
x_lamb = zoning["X_LAMB"]
x_lamb = x_lamb.to_numpy()
y_lamb = zoning["Y_LAMB"]
y_lamb = y_lamb.to_numpy()

# Project to correct CRS. 
inProj = Proj(init='epsg:31370')
outProj = Proj(init='epsg:4326')
x_centroids, y_centroids = transform(inProj,outProj,x_lamb,y_lamb)

# Add the centroids to the network. Relabelling the graph is required (see dyntapy demo for reason why)
network = add_centroids(network, x_centroids,y_centroids,k=1, method='link')
network = relabel_graph(network)
# show_network(network, notebook=True)

# Create OD graph
od_graph = od_graph_from_matrix(od_array,x_centroids,y_centroids) 

In [ ]:
# The network, zoning file and od matrix have been correctly loaded and linked by now. 
# This means that we can now run an assignment on the real demand! 
assignment = StaticAssignment(network,od_graph)
result = assignment.run('dial_b')
show_network(network, flows = result.flows, show_nodes=False)
# all static assignments return a result object that follows the same structure, see below
print(result.__dict__.keys())

In [ ]:
veh_hours = result.link_costs * result.flows
objective = sum(veh_hours)
print("Objective: %s" % objective)

2. HEEDS preparation

In [ ]:
# This way of working is useful because we only need to compute those files once,
# instead of in each iteration made by HEEDS. 
buffer = str(buffer)
HEEDS_path = HERE + os.path.sep + os.pardir + os.path.sep + os.pardir + os.path.sep + 'data_map' + os.path.sep + 'HEEDS_input'
network_path = HEEDS_path + os.path.sep + 'network_with_centroids' + os.path.sep + 'inelastic_' + city + '_' + buffer
od_graph_path = HEEDS_path + os.path.sep + 'od_graph' + os.path.sep + 'inelastic_' + city + '_' + buffer

with open(network_path, 'wb') as network_file:
    dump(network, network_file)
    print(f'network saved at f{network_path}')

with open(od_graph_path, 'wb') as f:
    dump(od_graph,f)
    print(f'od_graph saved at f{od_graph_path}')